In [2]:
cd '../'

/Users/andyvillanuevamartinez/Library/CloudStorage/GoogleDrive-villanuevaoct@gmail.com/My Drive/MAC/cursos/machine_learning/15_udacity_nanodegree_ds/11. capstone/sbux_recommendation


In [3]:
import numpy as np
import pandas as pd
import joblib

from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean

import warnings
warnings.filterwarnings("ignore")

In [4]:
best_channel_model = joblib.load('objects/best_channel.pkl')
spend_model = joblib.load('objects/spend.pkl')
cuts = np.load('objects/cuts_spend.npy')
ttc_offer = joblib.load('objects/ttc_offer.pkl')

In [5]:
scaler = MinMaxScaler()
vec = ['flag_mobile','flag_web','flag_social','difficulty','duration']

portfolio0 = pd.read_json('data/portfolio.json', orient = 'records', lines = True)
portfolio = portfolio0.copy()
portfolio['flag_mobile'] = portfolio.channels.astype(str).str.contains('mobile')*1
portfolio['flag_web'] = portfolio.channels.astype(str).str.contains('web')*1
portfolio['flag_social'] = portfolio.channels.astype(str).str.contains('social')*1
del portfolio['channels']
portfolio = portfolio[['id','offer_type'] + vec]
portfolio[vec] = scaler.fit_transform(portfolio[vec])
portfolio

,id,offer_type,flag_mobile,flag_web,flag_social,difficulty,duration
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,1.0,0.0,1.0,0.50,0.571429
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,1.0,1.0,1.0,0.50,0.285714
2,3f207df678b143eea3cee63160fa8bed,informational,1.0,1.0,0.0,0.00,0.142857
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,1.0,1.0,0.0,0.25,0.571429
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,0.0,1.0,0.0,1.00,1.000000
5,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,1.0,1.0,1.0,0.35,0.571429
6,fafdcd668e3743c1bb461111dcafc2a4,discount,1.0,1.0,1.0,0.50,1.000000
7,5a8bc65990b245e5a138643cd4eb9837,informational,1.0,0.0,1.0,0.00,0.000000
8,f19421c1d4aa40978ebb69ca19b0e20d,bogo,1.0,1.0,1.0,0.25,0.285714
9,2906b810c7d4411798c6938adc9daaa5,discount,1.0,1.0,0.0,0.50,0.571429


In [6]:
joblib.dump(portfolio, 'objects/portfolio_scaled.pkl', compress = True)

['objects/portfolio_scaled.pkl']

In [7]:
joblib.dump(scaler, 'objects/portfolio_scaler.pkl', compress = True)

['objects/portfolio_scaler.pkl']

In [8]:
import gradio as gr

In [9]:
def get_recommendation(gender, age, income):
    new_client = {'gender': gender, 'age': age, 'income': income}
    new_client_df = pd.DataFrame(new_client, index=[0])
    new_client_df[['flag_mobile','flag_web','flag_social']] = best_channel_model.predict(new_client_df) # mobile, web, social

    position = int(round(spend_model.predict(new_client_df)[0],0))
    if position >= 20:
        spend = cuts[18]
    elif position <= 1:
        spend = cuts[0]
    else:
        spend = (cuts[position - 1] + cuts[position - 2]) / 2
    new_client_df['difficulty'] = spend

    new_client_df['duration'] = ttc_offer.predict(new_client_df)[0]
    new_client_df[vec] = scaler.transform(new_client_df[vec])
    row = new_client_df[vec].values
    target_array = portfolio[vec].values
    distances = np.apply_along_axis(lambda x: euclidean(row, x), axis=1, arr=target_array)
    offer = portfolio0[portfolio0.index == np.argmin(distances)]
    offer = offer.T.reset_index()
    offer.columns = ['item', 'value']
    offer = offer[::-1].reset_index(drop = True)
    return offer

In [11]:
gender_input = gr.inputs.Radio(choices=['M', 'F'], label = 'Gender')
age_input = gr.inputs.Slider(minimum = 18, maximum = 120, step = 1, label = 'Age')
income_input = gr.inputs.Slider(minimum = 0, maximum = 500000, step = 1000, label = 'Income')
output = gr.DataFrame(headers=['Selected Offer'], label='Results')#gr.outputs.Textbox(label = 'Recommendation')

interface = gr.Interface(fn = get_recommendation, inputs = [gender_input, age_input, income_input], outputs = output, title = 'Recommendations ☕️',  allow_flagging = 'never')
interface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
